# Исследование игровой индустрии с 2000 по 2013 год

- Автор: Кокорин Максим
- Дата: 20.06.2025

### Цели и задачи проекта

Обработать и проанализировать данные о продажах и рейтингах компьютерных игр для дальнейшего написания статьи-обзора на основе полученной информации.

Задачи:
- предобработать данные,
- разделить игры на группы в зависимости от оценок пользователей и критиков,
- составить топ-7 платформ по количеству игр.

### Описание данных

Данные `new_games.csv` содержат информацию о продажах игр разных жанров и платформ, а также пользовательские и экспертные оценки игр:

- `Name` — название игры.
- `Platform` — название платформы.
- `Year of Release` — год выпуска игры.
- `Genre` — жанр игры.
- `NA sales` — продажи в Северной Америке (в миллионах проданных копий).
- `EU sales` — продажи в Европе (в миллионах проданных копий).
- `JP sales` — продажи в Японии (в миллионах проданных копий).
- `Other sales` — продажи в других странах (в миллионах проданных копий).
- `Critic Score` — оценка критиков (от 0 до 100).
- `User Score` — оценка пользователей (от 0 до 10).
- `Rating` — рейтинг организации ESRB (англ. Entertainment Software Rating Board). Эта ассоциация определяет рейтинг компьютерных игр и присваивает им подходящую возрастную категорию.

### Содержимое проекта


1. Загрузка данных и знакомство с ними
2. Проверка ошибок в данных и их предобработка
3. Фильтрация данных
4. Категоризация данных
5. Итоговый вывод

---

## 1. Загрузка данных и знакомство с ними

In [1]:
# Импортируем библиотку pandas
import pandas as pd

In [2]:
# Считываем файл с расширением csv в переменную df
df = pd.read_csv('new_games.csv')

In [3]:
# Выведем первые 5 строк датафрейма df
df.head()

,Name,Platform,Year of Release,Genre,NA sales,EU sales,JP sales,Other sales,Critic Score,User Score,Rating
0,Wii Sports,Wii,2006.0,Sports,41.36,28.96,3.77,8.45,76.0,8,E
1,Super Mario Bros.,NES,1985.0,Platform,29.08,3.58,6.81,0.77,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,15.68,12.76,3.79,3.29,82.0,8.3,E
3,Wii Sports Resort,Wii,2009.0,Sports,15.61,10.93,3.28,2.95,80.0,8,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,11.27,8.89,10.22,1.00,NaN,NaN,NaN


In [4]:
# Посмотрим общую информацию о датафрейме
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16956 entries, 0 to 16955
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Name             16954 non-null  object 
 1   Platform         16956 non-null  object 
 2   Year of Release  16681 non-null  float64
 3   Genre            16954 non-null  object 
 4   NA sales         16956 non-null  float64
 5   EU sales         16956 non-null  object 
 6   JP sales         16956 non-null  object 
 7   Other sales      16956 non-null  float64
 8   Critic Score     8242 non-null   float64
 9   User Score       10152 non-null  object 
 10  Rating           10085 non-null  object 
dtypes: float64(4), object(7)
memory usage: 1.4+ MB


Данные объёмом около 1,5 МБ представлены 11 столбцами, четыре из которых представлены типом `float64`, а 7 остальных — `object`. Названия столбцов соответствуют их содержимому, но написаны не в стиле snake case. Всего строк в датафрейме 16956, и 6 столбцов имеют пропуски (`Name`, `Year of Release`, `Genre`, `Critic Score`, `User Score`, `Rating`). Типы данных можно изменить у следующих столбцов: `Year of Release` на `datetime64`, так как речь идёт о дате; `Genre` и `Rating` на `category`, поскольку такой тип данных лучше отобразит смысл данных в этих столбцах; `EU sales`, `JP sales`, `User Score` на `float64` в связи с тем, что в этих столбцах представлены дробные числа. 

---

## 2.  Проверка ошибок в данных и их предобработка

### 2.1. Названия, или метки, столбцов датафрейма

In [5]:
# Выведем названия столбцов датафрейма
df.columns

Index(['Name', 'Platform', 'Year of Release', 'Genre', 'NA sales', 'EU sales',
       'JP sales', 'Other sales', 'Critic Score', 'User Score', 'Rating'],
      dtype='object')

In [6]:
# Создадим функцию для приведения названий в стиль snake case
def snake_case(x):
    x = x.replace(' ', '_')
    return x.lower()

In [7]:
# В переменную dm положим объект Series, который состоит из названий столбцов
dm = pd.Series(list(df.columns))
# Применим метод apply для применения функции и переименуем столбцы
df.columns = list(dm.apply(snake_case))
# Проверим внесенные изменения
df.head(1)

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating
0,Wii Sports,Wii,2006.0,Sports,41.36,28.96,3.77,8.45,76.0,8,E


### 2.2. Типы данных

Так как в столбце `year_of_release` содержится только число, он принял числовой тип данных, однако это число является годом выпуска игры. В столбцах `eu_sales`, `jp_sales`, `user_score` были найдены значения, отличные от типа `float64`, из-за чего данные в столбцах не были преобразованы к нужному типу автоматически. Столбцы `rating` и `genre` содержат строковый тип данных, поэтому они были преобразованы в object.

In [8]:
# Проверим уникальные записи в столбце
df['eu_sales'].unique()

array(['28.96', '3.58', '12.76', '10.93', '8.89', '2.26', '9.14', '9.18',
       '6.94', '0.63', '10.95', '7.47', '6.18', '8.03', '4.89', '8.49',
       '9.09', '0.4', '3.75', '9.2', '4.46', '2.71', '3.44', '5.14',
       '5.49', '3.9', '5.35', '3.17', '5.09', '4.24', '5.04', '5.86',
       '3.68', '4.19', '5.73', '3.59', '4.51', '2.55', '4.02', '4.37',
       '6.31', '3.45', '2.81', '2.85', '3.49', '0.01', '3.35', '2.04',
       '3.07', '3.87', '3.0', '4.82', '3.64', '2.15', '3.69', '2.65',
       '2.56', '3.11', '3.14', '1.94', '1.95', '2.47', '2.28', '3.42',
       '3.63', '2.36', '1.71', '1.85', '2.79', '1.24', '6.12', '1.53',
       '3.47', '2.24', '5.01', '2.01', '1.72', '2.07', '6.42', '3.86',
       '0.45', '3.48', '1.89', '5.75', '2.17', '1.37', '2.35', '1.18',
       '2.11', '1.88', '2.83', '2.99', '2.89', '3.27', '2.22', '2.14',
       '1.45', '1.75', '1.04', '1.77', '3.02', '2.75', '2.16', '1.9',
       '2.59', '2.2', '4.3', '0.93', '2.53', '2.52', '1.79', '1.3', '2.6',
   

In [9]:
df['jp_sales'].unique()

array(['3.77', '6.81', '3.79', '3.28', '10.22', '4.22', '6.5', '2.93',
       '4.7', '0.28', '1.93', '4.13', '7.2', '3.6', '0.24', '2.53',
       '0.98', '0.41', '3.54', '4.16', '6.04', '4.18', '3.84', '0.06',
       '0.47', '5.38', '5.32', '5.65', '1.87', '0.13', '3.12', '0.36',
       '0.11', '4.35', '0.65', '0.07', '0.08', '0.49', '0.3', '2.66',
       '2.69', '0.48', '0.38', '5.33', '1.91', '3.96', '3.1', '1.1',
       '1.2', '0.14', '2.54', '2.14', '0.81', '2.12', '0.44', '3.15',
       '1.25', '0.04', '0.0', '2.47', '2.23', '1.69', '0.01', '3.0',
       '0.02', '4.39', '1.98', '0.1', '3.81', '0.05', '2.49', '1.58',
       '3.14', '2.73', '0.66', '0.22', '3.63', '1.45', '1.31', '2.43',
       '0.7', '0.35', '1.4', '0.6', '2.26', '1.42', '1.28', '1.39',
       '0.87', '0.17', '0.94', '0.19', '0.21', '1.6', '0.16', '1.03',
       '0.25', '2.06', '1.49', '1.29', '0.09', '2.87', '0.03', '0.78',
       '0.83', '2.33', '2.02', '1.36', '1.81', '1.97', '0.91', '0.99',
       '0.95', '2.0'

In [10]:
df['user_score'].unique()

array(['8', nan, '8.3', '8.5', '6.6', '8.4', '8.6', '7.7', '6.3', '7.4',
       '8.2', '9', '7.9', '8.1', '8.7', '7.1', '3.4', '5.3', '4.8', '3.2',
       '8.9', '6.4', '7.8', '7.5', '2.6', '7.2', '9.2', '7', '7.3', '4.3',
       '7.6', '5.7', '5', '9.1', '6.5', 'tbd', '8.8', '6.9', '9.4', '6.8',
       '6.1', '6.7', '5.4', '4', '4.9', '4.5', '9.3', '6.2', '4.2', '6',
       '3.7', '4.1', '5.8', '5.6', '5.5', '4.4', '4.6', '5.9', '3.9',
       '3.1', '2.9', '5.2', '3.3', '4.7', '5.1', '3.5', '2.5', '1.9', '3',
       '2.7', '2.2', '2', '9.5', '2.1', '3.6', '2.8', '1.8', '3.8', '0',
       '1.6', '9.6', '2.4', '1.7', '1.1', '0.3', '1.5', '0.7', '1.2',
       '2.3', '0.5', '1.3', '0.2', '0.6', '1.4', '0.9', '1', '9.7'],
      dtype=object)

In [11]:
# Изменим типы данных 6 столбцов
df['year_of_release'] = pd.to_datetime(df['year_of_release'], format='%Y')
for column in ['eu_sales', 'jp_sales', 'user_score']:
    df[column] = pd.to_numeric(df[column], errors='coerce')
df['genre'] = df['genre'].astype('category')
# Проверим обновленные типы данных
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16956 entries, 0 to 16955
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   name             16954 non-null  object        
 1   platform         16956 non-null  object        
 2   year_of_release  16681 non-null  datetime64[ns]
 3   genre            16954 non-null  category      
 4   na_sales         16956 non-null  float64       
 5   eu_sales         16950 non-null  float64       
 6   jp_sales         16952 non-null  float64       
 7   other_sales      16956 non-null  float64       
 8   critic_score     8242 non-null   float64       
 9   user_score       7688 non-null   float64       
 10  rating           10085 non-null  object        
dtypes: category(1), datetime64[ns](1), float64(6), object(3)
memory usage: 1.3+ MB


### 2.3. Наличие пропусков в данных

In [12]:
# Найдем общее число пропусков в столбце sum и относительное число пропусков в столбце mean
round(df.isna().agg(['sum', 'mean']).transpose(), 4)

,sum,mean
name,2.0,0.0001
platform,0.0,0.0000
year_of_release,275.0,0.0162
genre,2.0,0.0001
na_sales,0.0,0.0000
eu_sales,6.0,0.0004
jp_sales,4.0,0.0002
other_sales,0.0,0.0000
critic_score,8714.0,0.5139
user_score,9268.0,0.5466


Пропуски присутствуют в следующих столбцах: `name` — 2 пропуска, `year_of_release` — 275 пропусков, `genre` — 2 пропуска, `eu_sales` — 6 пропусков, `jp_sales` — 4 пропуска, `critic_score` — 8714 пропусков, `user_score` — 9268 пропусков, `rating` — 6871 пропуск. Пропуски могли возникнуть из-за того, что данные в столбцах (`critic_score`, `user_score`, `rating`) не получилось найти из-за того, что игра была не очень популярна или быстро прекратила свое существование, но таких строк около 40 или больше 50%, поэтому их заменим на значения-индикаторы. В столбцах `name`, `year_of_release`, `genre` пропуски удалим, так как их немного (менее 2%) и для анализа важен год выпуска. Пропуски в столбцах `eu_sales` и `jp_sales` можно заменить на 0, поскольку в столбце `na_sales` пропусков у этих строк нет, а значит продажи были, скорее всего, только в Северной Америке.

In [13]:
# Заведём переменную str_del для подсчета удалённых строк
str_del = 0
# Сохраним текущее количество строк датафрейма
ln_old = len(df)

In [14]:
# Удаляем пропуски в трёх столбцах
df = df.dropna(subset=['name', 'year_of_release', 'genre'])
# Обновляем значение переменной str_del
str_del = ln_old - len(df)
# В двух столбцах заменим пропуски на 0
df['eu_sales'] = df['eu_sales'].fillna(0)
df['jp_sales'] = df['jp_sales'].fillna(0)
# В двух столбцах заменим пропуски на индикатор -1
df['critic_score'] = df['critic_score'].fillna(-1)
df['user_score'] = df['user_score'].fillna(-1)
# В столбце rating заменим пропуски на индикатор non_rating
df['rating'] = df['rating'].fillna('non_rating').astype('category')
# Проверим результат
round(df.isna().agg(['sum', 'mean']).transpose(), 4)

,sum,mean
name,0.0,0.0
platform,0.0,0.0
year_of_release,0.0,0.0
genre,0.0,0.0
na_sales,0.0,0.0
eu_sales,0.0,0.0
jp_sales,0.0,0.0
other_sales,0.0,0.0
critic_score,0.0,0.0
user_score,0.0,0.0


### 2.4. Явные и неявные дубликаты в данных

In [15]:
# Проверим число уникальных значений в 5 столбцах
for column in ['name', 'platform', 'year_of_release', 'genre', 'rating']: 
    print(f'{column}: {df[column].nunique()}')

name: 11426
platform: 31
year_of_release: 37
genre: 24
rating: 9


In [16]:
# Выведем отсортированный список уникальных названий игр
df['name'].sort_values().unique()

array([' Beyblade Burst', ' Fire Emblem Fates', " Frozen: Olaf's Quest",
       ..., 'uDraw Studio', 'uDraw Studio: Instant Artist',
       '¡Shin Chan Flipa en colores!'], dtype=object)

In [17]:
# Выведем отсортированный список уникальных названий платформ
df['platform'].sort_values().unique()

array(['2600', '3DO', '3DS', 'DC', 'DS', 'GB', 'GBA', 'GC', 'GEN', 'GG',
       'N64', 'NES', 'NG', 'PC', 'PCFX', 'PS', 'PS2', 'PS3', 'PS4', 'PSP',
       'PSV', 'SAT', 'SCD', 'SNES', 'TG16', 'WS', 'Wii', 'WiiU', 'X360',
       'XB', 'XOne'], dtype=object)

In [18]:
# Выведем отсортированный список уникальных названий жанров
df['genre'].sort_values().unique()

['ACTION', 'ADVENTURE', 'Action', 'Adventure', 'FIGHTING', ..., 'STRATEGY', 'Shooter', 'Simulation', 'Sports', 'Strategy']
Length: 24
Categories (24, object): ['ACTION', 'ADVENTURE', 'Action', 'Adventure', ..., 'Shooter', 'Simulation', 'Sports', 'Strategy']

In [19]:
# Выведем отсортированный список уникальных значений рейтингов организации ESRB
df['rating'].sort_values().unique()

['AO', 'E', 'E10+', 'EC', 'K-A', 'M', 'RP', 'T', 'non_rating']
Categories (9, object): ['AO', 'E', 'E10+', 'EC', ..., 'M', 'RP', 'T', 'non_rating']

In [20]:
# Удалим лишние пробелы и приведем к нижнему регистру названия игр
df['name'] = df['name'].str.strip().str.lower()
# Проверим количество уникальных значений
df['name'].nunique()

11426

In [21]:
# Удалим лишние пробелы и приведем к нижнему регистру названия жанров
df['genre'] = df['genre'].str.strip().str.lower()
df['genre'].nunique()

12

In [22]:
# Проверим текущее количество строк
len(df)

16679

In [23]:
# Обновим значение str_del добавлением явных дубликатов
str_del += df.duplicated().sum()

In [24]:
# Удалим явные дубликаты
df = df.drop_duplicates()
# Число строк в датафрейме после удаления дубликатов
len(df)

16444

Было найдено и удалено 235 строк-дубликатов.

In [25]:
# Выведем абсолютное число удалённых строк
print(str_del)
# Выведем относительное число удалённых строк в процентах
print(round((str_del / ln_old), 4) * 100)

512
3.02


В результате преобразований датафрейм был сокращён на 512 строк, что составляет 3% от их общего количества. Были удалены пропуски в трёх столбцах, нормализованы названия игр и жанров, удалено 235 строк-дубликатов, сохранены пропуски в столбцах `critic_score`, `user_score`, `rating`, так как они составляют более 40% данных в каждом столбце.

---

## 3. Фильтрация данных

In [26]:
# В переменную df_actual запишем датафрейм df с периодом данных с 2000 по 2013 год
df_actual = df[(df['year_of_release'].dt.year >= 2000) & (df['year_of_release'].dt.year <= 2013)]
# Проверим границы периода
df_actual['year_of_release'].agg(['min', 'max'])

min   2000-01-01
max   2013-01-01
Name: year_of_release, dtype: datetime64[ns]

---

## 4. Категоризация данных

In [27]:
# Разделим игры по оценкам пользователей на категории и запишем результат в новый столбец — group_user_score
df_actual['group_user_score'] = pd.cut(df_actual['user_score'], bins=[-1, 2, 7, 10], labels=['низкая оценка', 'средняя оценка', 'высокая оценка'])
# Проверим результат по первым 5 строкам
df_actual.head()

C:\Users\mokok\AppData\Local\Temp\ipykernel_1436\1935371172.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_actual['group_user_score'] = pd.cut(df_actual['user_score'], bins=[-1, 2, 7, 10], labels=['низкая оценка', 'средняя оценка', 'высокая оценка'])


,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating,group_user_score
0,wii sports,Wii,2006-01-01,sports,41.36,28.96,3.77,8.45,76.0,8.0,E,высокая оценка
2,mario kart wii,Wii,2008-01-01,racing,15.68,12.76,3.79,3.29,82.0,8.3,E,высокая оценка
3,wii sports resort,Wii,2009-01-01,sports,15.61,10.93,3.28,2.95,80.0,8.0,E,высокая оценка
6,new super mario bros.,DS,2006-01-01,platform,11.28,9.14,6.50,2.88,89.0,8.5,E,высокая оценка
7,wii play,Wii,2006-01-01,misc,13.96,9.18,2.93,2.84,58.0,6.6,E,средняя оценка


In [28]:
# Разделим игры по оценкам критиков на категории и запишем результат в новый столбец — group_critic_score
df_actual['group_critic_score'] = pd.cut(df_actual['critic_score'], bins=[-1, 29, 79, 100], labels=['низкая оценка', 'средняя оценка', 'высокая оценка'])
df_actual.head()

C:\Users\mokok\AppData\Local\Temp\ipykernel_1436\3144232140.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_actual['group_critic_score'] = pd.cut(df_actual['critic_score'], bins=[-1, 29, 79, 100], labels=['низкая оценка', 'средняя оценка', 'высокая оценка'])


,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating,group_user_score,group_critic_score
0,wii sports,Wii,2006-01-01,sports,41.36,28.96,3.77,8.45,76.0,8.0,E,высокая оценка,средняя оценка
2,mario kart wii,Wii,2008-01-01,racing,15.68,12.76,3.79,3.29,82.0,8.3,E,высокая оценка,высокая оценка
3,wii sports resort,Wii,2009-01-01,sports,15.61,10.93,3.28,2.95,80.0,8.0,E,высокая оценка,высокая оценка
6,new super mario bros.,DS,2006-01-01,platform,11.28,9.14,6.50,2.88,89.0,8.5,E,высокая оценка,высокая оценка
7,wii play,Wii,2006-01-01,misc,13.96,9.18,2.93,2.84,58.0,6.6,E,средняя оценка,средняя оценка


In [29]:
# Посчитаем количество игр по категориям group_user_score
df_actual.groupby('group_user_score', observed=True)['name'].count()

group_user_score
низкая оценка       47
средняя оценка    2351
высокая оценка    4085
Name: name, dtype: int64

In [30]:
# Посчитаем количество игр по категориям group_critic_score
df_actual.groupby('group_critic_score', observed=True)['name'].count()

group_critic_score
низкая оценка       55
средняя оценка    5422
высокая оценка    1692
Name: name, dtype: int64

In [31]:
# Выделим топ-7 платформ по количеству игр
pd.DataFrame(df_actual.groupby('platform')['name'].count().sort_values(ascending=False).head(7)).reset_index()

,platform,name
0,PS2,2127
1,DS,2120
2,Wii,1275
3,PSP,1180
4,X360,1121
5,PS3,1087
6,GBA,811


---

## 5. Итоговый вывод

Данные файла `new_games.csv` объёмом около 1,5 МБ были представлены 11 столбцами, четыре из которых представлены типом `float64`, а 7 остальных — `object`. Названия столбцов соответствуют их содержимому, но написаны не в стиле snake case. Всего строк в датафрейме 16956, и 6 столбцов имели пропуски.

Типы данных были изменены у столбцов: `Year of Release` на `datetime64`, `Genre` и `Rating` на `category`, `EU sales`, `JP sales`, `User Score` на `float64`.

Пропуски были заменены индикатором в столбцах `critic_score`, `user_score`, `rating`, удалены в `name`, `year_of_release`, `genre`, заменены значением 0 в `eu_sales` и `jp_sales`. Удалено 235 строк-дубликатов.

В результате мы получили датасет за период с 2000 по 2013 год, в который добавили два новых столбца с типом данных `category` — `group_user_score` и `group_critic_score`. Мы узнали, что пользователи ставят играм высокую оценку почти в 2,5 раза чаще, чем критики (1692 игры у критиков и 4085 игр у пользователей получили высокую оценку), среднюю оценку критики ставят больше, чем игроки (5422 у критиков и 2351 у пользователей), а вот низкая оценка выставлялась почти одинаково редко (55 у критиков и 47 у пользователей). Также составили топ-7 платформ по количеству игр. Эта информация пригодится для написания статьи.